In [3]:
import gradio as gr
import pandas as pd
import mlflow
import numpy as np
import mlflow.pyfunc
import xgboost as xgb
import mlflow.xgboost
import pickle
from gradio_functions import *
import matplotlib.pyplot as plt
import plotly.express as px
from model_functions import *
import html

mlflow.set_tracking_uri("http://localhost:5000")

In [4]:
with gr.Blocks() as demo:
    with gr.Tab(label="Immopreisrechner"):
        gr.Markdown(
            """
        # Geben Sie hier an, welche Merkmale Ihre Immobilie besitzt 🏠
        """
        )
        with gr.Row():
            feature_zip = gr.Dropdown(
                label="Postleitzahl",
                choices=[
                    "97070 Würzburg-Altstadt",
                    "97072 Würzburg-Sanderau",
                    "97074 Würzburg-Frauenland",
                    "97076 Würzburg-Lengfeld",
                    "97078 Würzburg-Lindleinsmühle",
                    "97080 Würzburg-Dürrbachtal",
                    "97082 Würzburg-Steinbachtal",
                    "97084 Würzburg-Heuchelhof",
                    "97204 Höchberg",
                    "97209 Veitshöchheim",
                    "97218 Gerbrunn",
                    "97222 Rimpar",
                    "97228 Rottendorf",
                    "97234 Reichenberg",
                    "97236 Randersacker",
                    "97246 Eibelstadt",
                    "97249 Eisingen",
                    "97250 Erlabrunn",
                    "97261 Güntersleben",
                    "97270 Kist",
                    "97288 Theilheim",
                    "97297 Waldbüttelbrunn",
                    "97299 Zell am Main",
                ],
                value="97070 Würzburg-Altstadt",
            )
        with gr.Row():
            feature_squrmeter = gr.Number(label="Wohnfläche [qm]", value=79)
        with gr.Row():
            feature_rooms = gr.Number(label="Anzahl der Zimmer", value=3)
        with gr.Row():
            features_altbau = gr.Checkbox(label="Altbau (bis 1945)")
            feature_balkon = gr.Checkbox(label="Balkon")
            feature_ba = gr.Checkbox(label="Barrierefrei")
            feature_dachgeschoss = gr.Checkbox(label="Dachgeschoss")
            feature_einbaukueche = gr.Checkbox(label="Einbauküche")
            feature_neubau = gr.Checkbox(label="Neubau")
            feature_parkett = gr.Checkbox(label="Parkett")
            feature_stellplatz = gr.Checkbox(label="Stellplatz")
            feature_badwc_getrennt = gr.Checkbox(label="Bad/WC getrennt")
            feature_personenaufzug = gr.Checkbox(label="Personenaufzug")
            feature_garten = gr.Checkbox(label="Garten")
            feature_garage = gr.Checkbox(label="Garage")
            feature_renoviert = gr.Checkbox(label="Renoviert")
            feature_terrasse = gr.Checkbox(label="Terrasse")
            feature_wanne = gr.Checkbox(label="Badewanne")
            feature_zentralheizung = gr.Checkbox(label="Zentralheizung")
            feature_abstellraum = gr.Checkbox(label="Abstellraum")
            feature_fernwaerme = gr.Checkbox(label="Fernwärme")
            feature_fussbodenheitzung = gr.Checkbox(label="Fußbodenheizung")
            feature_gartenmitbenutzung = gr.Checkbox(label="Gartenmitbenutzung")
            feature_kellerabteil = gr.Checkbox(label="Kellerabteil")

        with gr.Accordion("erweiterte Optionen", open=False):
            erklärung = gr.Checkbox(label="XGBoost vorhersagen")

        btn = gr.Button("Berechne deinen Preis")
        output_html = gr.HTML(label="Dein Preis")
        btn.click(
            fn=trigger_actions,
            inputs=[
                feature_squrmeter,
                feature_zip,
                feature_rooms,
                features_altbau,
                feature_balkon,
                feature_ba,
                feature_dachgeschoss,
                feature_einbaukueche,
                feature_neubau,
                feature_parkett,
                feature_stellplatz,
                feature_badwc_getrennt,
                feature_personenaufzug,
                feature_garten,
                feature_garage,
                feature_renoviert,
                feature_terrasse,
                feature_wanne,
                feature_zentralheizung,
                feature_abstellraum,
                feature_fernwaerme,
                feature_fussbodenheitzung,
                feature_gartenmitbenutzung,
                feature_kellerabteil,
                erklärung,
            ],
            # gebe mir den output der funktion in einem number gradio widget aus
            outputs=[output_html],
        )

    with gr.Tab(label="Admin"):
        gr.Markdown(
            """
        # Admin-Seite
        Hier können Sie Ihre Modelle nachtrainieren und die verschiedenen Modelle vergleichen.
        """
        )
        gr.Markdown("## Datengrundlage erkunden und verstehen:")
        link = "https://michaelseitz98.github.io/enterprise-ai-project/eda-wue-rent-all.html"
        description = (
            "Datenanalyse des Ausgangsdatensatz für Würzburger Wohnungen (Miete), EDA"
        )
        html_code = f'<a href="{html.escape(link)}" font-size: 16px;">{html.escape(description)}</a>'
        gr.HTML(html_code)
        link = (
            "https://michaelseitz98.github.io/enterprise-ai-project/eda-wue-houses.html"
        )
        description = "Jetzt anschauen: Datenanalyse des Ausgangsdatensatz Würzburger Häuser (Kauf), EDA"
        html_code = f'<a href="{html.escape(link)}" font-size: 16px;">{html.escape(description)}</a>'
        gr.HTML(html_code)
        gr.HTML("<hr>")

        gr.Markdown("## Dynamisches Nachtraining")
        gr.Markdown("Wählen Sie die Modelle aus, die Sie nachtrainieren möchten und miteinander vergleichen möchten")

        with gr.Row():
            xgb = gr.Checkbox(label="xgb")
            rf = gr.Checkbox(label="rf")
            linear = gr.Checkbox(label="linear", value=True)
            lasso = gr.Checkbox(label="lasso")
            ridge = gr.Checkbox(label="ridge")
            elasticnet = gr.Checkbox(label="elasticnet")
            baseline = gr.Checkbox(label="baseline-rent", value=True)

        model_list = []
        if xgb:
            model_list.append("xgb")
        if rf:
            model_list.append("rf")
        if lasso:
            model_list.append("lasso")
        if ridge:
            model_list.append("ridge")
        if elasticnet:
            model_list.append("elasticnet")
        if baseline:
            model_list.append("baseline-rent")

        limit = gr.Number(label="Anzahl der zu scrapenden Seiten ", value=1, maximum=10)

        nachtraining = gr.Button("Aktuelle Daten Scrapen, Modelle nachtrainieren und evaluieren")
        
        df_results = gr.Dataframe(label="Ergebnisse Nachtraining")
        output_plot = gr.Plot(visible=False)

        nachtraining.click(fn=gradio_retrain_with_added_data, inputs=[xgb, ridge, rf, elasticnet, linear, lasso, baseline, limit], outputs=[df_results, output_plot])

demo.launch(debug=True)

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


https://www.immowelt.de/liste/wuerzburg/wohnungen/mieten?d=true&r=10&sd=DESC&sf=RELEVANCE&sp=1
started
Retraining data successfully scraped.
Retraining data successfully written to excel under data/retrain_train_data.xslx
Done with raw preprocessing.
old shape of train_recent (108, 47)
Index(['Object_price', 'LivingSpace', 'Rooms', 'ConstructionYear', 'ZipCode',
       'EstateType', 'DistributionType', 'abstellraum', 'altbau_(bis_1945)',
       'balkon', 'barriefrei', 'dachgeschoss', 'dsl', 'dusche', 'einbaukueche',
       'elektro', 'erdgeschoss', 'etagenheizung', 'fenster', 'fern', 'ferne',
       'fliesen', 'frei', 'fussbodenheizung', 'gaestewc', 'garten',
       'gartennutzung', 'gas', 'gepflegt', 'kelleranteil', 'kunststoff',
       'kunststofffenster', 'luftwp', 'neubau', 'offene_kueche', 'parkett',
       'pellet', 'personenaufzug', 'reinigung', 'renoviert',
       'rollstuhlgerecht', 'sat', 'speisekammer', 'stellplatz',
       'teilweise_moebliert', 'teppich', 'terrasse', 'tief

2023/07/15 18:17:17 INFO mlflow.tracking.fluent: Experiment with name 'retraining_2023-07-15_18-17' does not exist. Creating a new experiment.
2023/07/15 18:17:25 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Users\michi\Anaconda3\envs\enterpriseai2\lib\site-packages\mlflow\data\pandas_dataset.py:116: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for mor

LINEAR------


2023/07/15 18:17:29 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Users\michi\Anaconda3\envs\enterpriseai2\lib\site-packages\mlflow\models\signature.py:144: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details."
2023/07/15 18:17:51 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Users\michi\Anaconda3\envs\e

Training linear model done...
---EVALUATION AND LOGGING TO MLFLOW------ linear


2023/07/15 18:17:53 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Users\michi\Anaconda3\envs\enterpriseai2\lib\site-packages\mlflow\data\pandas_dataset.py:116: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details."
2023/07/15 18:17:56 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Users\michi\Anaconda3\env

BASELINE-RENT------
Extrcated rental price per square meter via scraper: 11.21
Average rental price per sqm: 11.21
Baseline Mae: 285.5564
Baseline Mae: 285.5564
Baseline MSE: 140135.41872988813
Baseline R2 Score: 0.5802888162379594
Training baseline-rent model done...
---EVALUATION AND LOGGING TO MLFLOW------ baseline-rent


c:\Users\michi\enterpriseAI_michi\enterprise-ai-project\immowelt_price_guide\model_functions.py:616: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



Done with retraining:             model         mae            mse        r2    mae_test  \
0         linear  256.158912  120536.985958  0.638987  202.846712   
1  baseline-rent  285.556400  140135.418730  0.580289  237.832650   

       mse_test   r2_test   mae_train     mse_train  r2_train  
0  79008.886992  0.731956  135.969284  29663.754251  0.891478  
1  92491.552430  0.686215         NaN           NaN       NaN  
Save results to excel
Done with saving results to excel
Done with plotting:  Figure({
    'data': [{'alignmentgroup': 'True',
              'hovertemplate': 'model=%{x}<br>mae=%{y}<extra></extra>',
              'legendgroup': '',
              'marker': {'color': '#636efa', 'pattern': {'shape': ''}},
              'name': '',
              'offsetgroup': '',
              'orientation': 'v',
              'showlegend': False,
              'textposition': 'auto',
              'type': 'bar',
              'x': array(['linear', 'baseline-rent'], dtype=object),
          

In [ ]:
def getPlot (result_df): 
    result_df =  pd.read_excel("retraining_results.xlsx")
    plot = px.bar(
            result_df,
            x="model",
            y="mae",
            title="Modellperformance mit erweitereten Trainingsdaten"
        )   

    plot.show()
    return plot

with gr.Blocks() as demo:
    df = pd.read_excel("retraining_results.xlsx")

    btn = gr.Button("Berechne deinen Preis")
    btn.click(getPlot, outputs=[gr.Plot()])




demo.launch(debug=True)

c:\Users\michi\Anaconda3\envs\enterpriseai2\lib\site-packages\gradio\utils.py:766: UserWarning:

Expected 1 arguments for function <function getPlot at 0x0000017976215160>, received 0.

c:\Users\michi\Anaconda3\envs\enterpriseai2\lib\site-packages\gradio\utils.py:770: UserWarning:

Expected at least 1 arguments for function <function getPlot at 0x0000017976215160>, received 0.



Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


c:\Users\michi\Anaconda3\envs\enterpriseai2\lib\site-packages\gradio\helpers.py:661: UserWarning:

Unexpected argument. Filling with None.



c:\Users\michi\Anaconda3\envs\enterpriseai2\lib\site-packages\gradio\helpers.py:661: UserWarning:

Unexpected argument. Filling with None.



c:\Users\michi\Anaconda3\envs\enterpriseai2\lib\site-packages\gradio\helpers.py:661: UserWarning:

Unexpected argument. Filling with None.



Keyboard interruption in main thread... closing server.
